# **Imports**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from datetime import datetime
import re

# **Functions**

In [ ]:
#geodesic is important

In [ ]:
def process(data):
  data=data[data['Timestamp'].str.len()>10]
  

  data['Timestamp']=pd.to_datetime(data['Timestamp'], format='%Y-%m-%d %H:%M:%S' ,utc=True)

  data['day'] = data['Timestamp'].dt.day
  data['hour'] = data['Timestamp'].dt.hour
  # data['minute'] = data['Timestamp'].dt.minute

  data=data[data.RadioConnectionType =='Mobile']
  data=data[data.RadioMobileDataEnabled =='Enabled']
  data=data[data.RadioNetworkGeneration =='4G']

  
  data=data.drop(['Country'], axis=1)
  data=data.drop(['RadioConnectionType'], axis=1)
  data=data.drop(['RadioMobileDataEnabled'], axis=1)
  data=data.drop(['Timestamp'], axis=1)
  data=data.drop(['RadioNetworkGeneration'], axis=1)

  return data




def process_rsrp(data):
  data=data[(data['RSRP']  <=-40) & (data['RSRP'] >=-140)] 
  data['DeviceManufacturer']=data['DeviceManufacturer'].str.upper()
  data['DeviceName']=data['DeviceName'].str.upper()

  return data




def treatoutliers(df=None, columns=None, factor=1.5, method='IQR', treatment='cap'):

    for column in columns:
        if method == 'STD':
            permissable_std = factor * df[column].std()
            col_mean = df[column].mean()
            floor, ceil = col_mean - permissable_std, col_mean + permissable_std
        elif method == 'IQR':
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            floor, ceil = Q1 - factor * IQR, Q3 + factor * IQR
#         print(floor, ceil)
        if treatment == 'remove':
            print(treatment, column)
            df = df[(df[column] >= floor) & (df[column] <= ceil)]
            # link for   https://www.geeksforgeeks.org/numpy-clip-in-python/
            # clip to make all the data between the q1 and q3 and not make the data in outliers 
        elif treatment == 'cap':
            print(treatment, column)
            df[column] = df[column].clip(floor, ceil)

    return df
def treat_missing(data,cat_col ,num_col):#'most_frequent'
  imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
  imputer = imp_mean.fit(data[cat_col])
  data[cat_col]= imputer.transform(data[cat_col])

  imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
  imputer = imp_mean.fit(data[num_col])
  data[num_col]= imputer.transform(data[num_col])

  data = pd.concat([data[cat_col], data[num_col]],axis=1)
  return data


# **Read Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Task/TrafficVolume.xlsx')
rsrp = pd.read_csv('/content/drive/MyDrive/Task/RSRP.csv')

In [ ]:
rsrp=rsrp.sample(900000)

## **process in data and time**

In [ ]:
data=process(data)
rsrp=process(rsrp)
rsrp=process_rsrp(rsrp)

<ipython-input-3-e218148652e2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Timestamp']=pd.to_datetime(data['Timestamp'], format='%Y-%m-%d %H:%M:%S' ,utc=True)
<ipython-input-3-e218148652e2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['day'] = data['Timestamp'].dt.day
<ipython-input-3-e218148652e2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [ ]:
# rsrp['RSRP'].value_counts()
# rsrp['RSRP'].unique()


# **Missing value**

In [ ]:
cols_with_missing = [col for col in data.columns if data[col].isnull().any()]

print(cols_with_missing)

                    

cols_with_missing = [col for col in rsrp.columns if rsrp[col].isnull().any()]
                     
print(cols_with_missing)




                
                    

[]
[]


# No missing values

# **duplicate data**

In [ ]:
data=data.drop_duplicates()
# data.shape
# data

rsrp=rsrp.drop_duplicates()
# rsrp.shape
# rsrp

# **Merge data**

In [ ]:
merge_data= pd.merge(data, rsrp,on=["day","hour","LocationLatitude","LocationLongitude","RadioOperatorName"])

# merge_data=data.combine_first(rsrp)

cat_coumns = [cname for cname in merge_data.columns if merge_data[cname].dtype == "object"]
                 
num_columns  = [cname for cname in merge_data.columns if 
               merge_data[cname].dtype in ['int64', 'float64']]

                     
merge_data=treat_missing(merge_data,cat_coumns,num_columns)
merge_data.shape
merge_data

,RadioOperatorName,TrafficDirection,DeviceManufacturer,DeviceName,LocationLatitude,LocationLongitude,TrafficVolume,day,hour,RSRP
0,Operator C,Uplink,SAMSUNG,GALAXY A50,24.828726,46.81041,0.140186,1.0,21.0,-62.0
1,Operator C,Uplink,SAMSUNG,GALAXY A50,24.828726,46.81041,0.140186,1.0,21.0,-67.0
2,Operator C,Uplink,SAMSUNG,GALAXY A50,24.828726,46.81041,0.140186,1.0,21.0,-64.0
3,Operator C,Uplink,SAMSUNG,GALAXY A50,24.828726,46.81041,0.140186,1.0,21.0,-66.0
4,Operator C,Uplink,SAMSUNG,GALAXY A50,24.828726,46.81041,0.140186,1.0,21.0,-65.0
...,...,...,...,...,...,...,...,...,...,...
33748,Operator A,Uplink,HMD GLOBAL,NOKIA 6.1 PLUS,24.671516,46.71441,1.017016,2.0,17.0,-94.0
33749,Operator A,Uplink,HMD GLOBAL,NOKIA 6.1 PLUS,24.671516,46.71441,1.017016,2.0,17.0,-76.0
33750,Operator A,Downlink,HMD GLOBAL,NOKIA 6.1 PLUS,24.671516,46.71441,2.526154,2.0,17.0,-94.0
33751,Operator A,Downlink,HMD GLOBAL,NOKIA 6.1 PLUS,24.671516,46.71441,2.526154,2.0,17.0,-76.0


In [ ]:
merge_data.to_csv("data.csv")

In [ ]:
rsrp.to_csv("rsrp.csv")
data.to_csv("traffic_vplume.csv")


In [ ]:
data.shape

(102529, 7)

In [ ]:
rsrp.shape

(523390, 8)

In [ ]:
merge_data.shape

(33753, 10)